## create  auth_config

In [11]:
import requests
from typing import Dict, Any, Optional, List

COMPOSIO_BASE_URL = "https://backend.composio.dev/api/v3"

def create_auth_config(
    composio_api_key: str,
    toolkit_slug: str,
    auth_type: str = "OAUTH2",  # OAUTH2 | BEARER_TOKEN | API_KEY
    bearer_token: Optional[str] = None,
    api_key_secret: Optional[str] = None,
    scopes: Optional[List[str]] = None,
    config_name: Optional[str] = None
) -> Dict[str, Any]:
    """
    Create an Auth Config in Composio depending on the provided credentials.

    - If only composio_api_key is given -> OAUTH2 (managed)
    - If bearer_token is provided -> BEARER_TOKEN
    - If api_key_secret is provided -> API_KEY
    """

    url = f"{COMPOSIO_BASE_URL}/auth_configs"
    headers = {"x-api-key": composio_api_key, "Content-Type": "application/json"}

    if not config_name:
        config_name = f"{toolkit_slug.capitalize()} Auth Config"

    # Determine type automatically
    if bearer_token:
        auth_type = "BEARER_TOKEN"
    elif api_key_secret:
        auth_type = "API_KEY"
    else:
        auth_type = "OAUTH2"

    # Prepare payload depending on type
    if auth_type == "OAUTH2":
        data = {
            "toolkit": {"slug": toolkit_slug},
            "auth_config": {
                "type": "use_composio_managed_auth",
                "auth_scheme": "OAUTH2",
                "scopes": scopes or [],
                "name": config_name,
            },
        }
    elif auth_type == "BEARER_TOKEN":
        data = {
            "toolkit": {"slug": toolkit_slug},
            "auth_config": {
                "type": "user_provided_auth",
                "auth_scheme": "BEARER_TOKEN",
                "name": config_name,
                "credentials": {
                    "bearer_token": bearer_token
                },
            },
        }
    elif auth_type == "API_KEY":
        data = {
            "toolkit": {"slug": toolkit_slug},
            "auth_config": {
                "type": "user_provided_auth",
                "auth_scheme": "API_KEY",
                "name": config_name,
                "credentials": {
                    "api_key": api_key_secret
                },
            },
        }
    else:
        raise ValueError(f"❌ Unsupported auth type: {auth_type}")

    # Send request
    response = requests.post(url, json=data, headers=headers)
    response.raise_for_status()
    return response.json()


# Example Usage
if __name__ == "__main__":
    COMPOSIO_API_KEY = "ak_GepAqzYQ6d6WggBF_zot"  # replace with your Composio API key

    # Case 1: Just OAUTH2 (Google Sheets for example)
    result = create_auth_config(composio_api_key=COMPOSIO_API_KEY, toolkit_slug="slack")
    print("✅ OAUTH2 Config:", result)

    # Case 2: With Bearer Token
    # result = create_auth_config(composio_api_key=COMPOSIO_API_KEY, toolkit_slug="twitter", bearer_token="your_bearer_token")

    # Case 3: With API Key
    # result = create_auth_config(composio_api_key=COMPOSIO_API_KEY, toolkit_slug="notion", api_key_secret="your_notion_api_key")


✅ OAUTH2 Config: {'toolkit': {'slug': 'slack'}, 'auth_config': {'id': 'ac_XlU4Aq9rZOq5', 'auth_scheme': 'OAUTH2', 'is_composio_managed': True, 'restrict_to_following_tools': []}}


## delete auth config

In [22]:
import requests

def delete_auth_config(nanoid: str, api_key: str):
    """
    Delete an authentication configuration from Composio API with detailed debug.

    :param nanoid: The ID of the auth config to delete
    :param api_key: Your Composio API key
    :return: JSON response from API or None if request fails
    """
    url = f"https://backend.composio.dev/api/v3/auth_configs/{nanoid}"
    headers = {
        "x-api-key": api_key
    }

    try:
        print(f"🔹 Sending DELETE request to: {url}")
        response = requests.delete(url, headers=headers)

        print(f"🔹 Status Code: {response.status_code}")
        print(f"🔹 Response Headers: {response.headers}")
        print(f"🔹 Response Text: {response.text}")

        if response.ok:
            print(f"✅ Auth config {nanoid} deleted successfully.")
            return response.json()
        else:
            print(f"❌ Failed to delete auth config {nanoid}")
            # Extra debug info
            if response.status_code == 401:
                print("⚠️ Unauthorized: Check your API key.")
            elif response.status_code == 403:
                print("⚠️ Forbidden: API key does not have delete permissions.")
            elif response.status_code == 404:
                print("⚠️ Not Found: Check if the nanoid is correct.")
            return None

    except requests.exceptions.RequestException as e:
        print("⚠️ Request Exception:", str(e))
        return None


# 🧪 Example usage
if __name__ == "__main__":
    YOUR_API_KEY = "ak_GepAqzYQ6d6WggBF_zot"  # <-- Replace with your API key
    NANOID_TO_DELETE = "ac_-F21CVJE32ak"             # <-- Replace with the config ID
    result = delete_auth_config(NANOID_TO_DELETE, YOUR_API_KEY)
    print("Delete Result:", result)


🔹 Sending DELETE request to: https://backend.composio.dev/api/v3/auth_configs/ac_-F21CVJE32ak
🔹 Status Code: 200
🔹 Response Headers: {'Date': 'Tue, 16 Sep 2025 06:00:33 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Server': 'cloudflare', 'CF-Ray': '97fe24e2c94151de-DEL', 'CF-Cache-Status': 'DYNAMIC', 'Cache-Control': 'public, max-age=0, must-revalidate', 'Content-Encoding': 'gzip', 'Strict-Transport-Security': 'max-age=63072000', 'Vary': 'Origin, accept-encoding', 'access-control-allow-credentials': 'true', 'Cf-Placement': 'local-', 'x-matched-path': '/api/v3/auth_configs/[nanoid]', 'x-powered-by': 'composio tech team', 'x-ratelimit': '20000', 'x-ratelimit-remaining': '19948', 'x-ratelimit-window-size': '600s', 'x-request-id': 'b3228d88-630a-4274-b677-0beb3f7d1edd', 'x-vercel-cache': 'MISS', 'x-vercel-id': 'bom1::iad1::sfmnt-1758002432893-3cc1cf8c41f6'}
🔹 Response Text: {"success":true,"message":"Successfully deleted auth config"

## update auth config 

In [13]:
import requests
import json

BASE_URL = "https://backend.composio.dev/api/v3"

def update_auth_config(nanoid: str, api_key: str, new_credentials: dict):
    """
    Update credentials and fetch the updated client_id and client_secret to verify change.
    """
    headers = {"x-api-key": api_key}

    # Step 1: Get current type
    try:
        resp = requests.get(f"{BASE_URL}/auth_configs/{nanoid}", headers=headers)
        resp.raise_for_status()
        current_config = resp.json()
        current_type = current_config.get("type", "default")
    except requests.exceptions.RequestException as e:
        print(f"❌ Failed to fetch current config: {e}")
        return None

    # Step 2: Update credentials
    payload = {
        "type": current_type,  # must include type
        "credentials": new_credentials
    }
    headers["Content-Type"] = "application/json"

    try:
        resp = requests.patch(f"{BASE_URL}/auth_configs/{nanoid}", headers=headers, json=payload)
        if resp.ok:
            print("✅ Credentials updated successfully")
        else:
            print(f"❌ Update failed: {resp.status_code} - {resp.text}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"❌ Network error during update: {e}")
        return None

    # Step 3: Fetch updated config to verify
    try:
        resp = requests.get(f"{BASE_URL}/auth_configs/{nanoid}", headers={"x-api-key": api_key})
        resp.raise_for_status()
        updated_config = resp.json()
        creds = updated_config.get("credentials", {})
        client_id = creds.get("client_id")
        client_secret = creds.get("client_secret")
        print(f"🔹 Updated client_id: {client_id}")
        print(f"🔹 Updated client_secret: {client_secret}")
        return updated_config
    except requests.exceptions.RequestException as e:
        print(f"❌ Failed to fetch updated config: {e}")
        return None


# -----------------------------
# Example usage
# -----------------------------
if __name__ == "__main__":
    API_KEY = "ak_GepAqzYQ6d6WggBF_zot"
    NANOID = "ac_XlU4Aq9rZOq5"

    new_credentials = {
        "client_id": "123456daljeet1",
        "client_secret": "123456daljeet123456daljeet1"
    }

    # Call the update function and store the result
    updated_config = update_auth_config(NANOID, API_KEY, new_credentials)

    # Optionally access the updated credentials programmatically
    if updated_config:
        creds = updated_config.get("credentials", {})
        client_id = creds.get("client_id")
        client_secret = creds.get("client_secret")
        print("\n✅ Verification complete:")
        print(f"Client ID: {client_id}")
        print(f"Client Secret: {client_secret}")


✅ Credentials updated successfully
🔹 Updated client_id: ****
🔹 Updated client_secret: ****

✅ Verification complete:
Client ID: ****
Client Secret: ****


## google connect

In [14]:
import requests
import json
import uuid
import webbrowser
import time

API_KEY = "ak_GepAqzYQ6d6WggBF_zot"   # 🔑 replace with your key
BASE_URL = "https://backend.composio.dev/api/v3"

HEADERS = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json"
}

def connect_gmail(auth_config_id: str):
    """Initiate Gmail OAuth connection and open browser"""
    url = f"{BASE_URL}/connected_accounts/link"
    # user_id = str(uuid.uuid4())  # 👈 yaha tum apna app user_id bhi de sakte ho
    user_id="daljeet44"
    payload = {"auth_config_id": auth_config_id, "user_id": user_id}

    r = requests.post(url, headers=HEADERS, json=payload, timeout=20)
    r.raise_for_status()
    data = r.json()

    redirect_url = data.get("redirect_url") or data.get("redirectUrl")
    connected_account_id = data.get("connected_account_id")

    print("✅ Gmail connect flow initiated.")
    print("🔗 Redirect URL:", redirect_url)
    if redirect_url:
        webbrowser.open(redirect_url)

    return connected_account_id

def poll_connection_status(connected_account_id: str):
    """Poll connection status until CONNECTED"""
    url = f"{BASE_URL}/connected_accounts/{connected_account_id}"
    for i in range(20):
        r = requests.get(url, headers=HEADERS, timeout=15)
        if not r.ok:
            print("⚠️ Failed to fetch status:", r.text)
            break
        status = r.json().get("status")
        print(f"⏳ Attempt {i+1}: Status =", status)
        if status and status.upper() in ("CONNECTED", "ACTIVE"):
            print("🎉 Gmail Connected Successfully!")
            return True
        elif status and status.upper() in ("FAILED", "ERROR"):
            print("❌ Gmail Connection Failed.")
            return False
        time.sleep(3)
    print("⌛ Timeout waiting for connection.")
    return False

if __name__ == "__main__":
    # 👇 Yaha apna Gmail Auth Config ID daalo
    AUTH_CONFIG_ID = "ac_DN9ql2I64yrm"  
    
    connected_account_id = connect_gmail(AUTH_CONFIG_ID)
    if connected_account_id:
        poll_connection_status(connected_account_id)


✅ Gmail connect flow initiated.
🔗 Redirect URL: https://connect.composio.dev/link/lk_D32_RSm7HvFG
⏳ Attempt 1: Status = INITIATED
⏳ Attempt 2: Status = INITIATED
⏳ Attempt 3: Status = INITIATED
⏳ Attempt 4: Status = INITIATED
⏳ Attempt 5: Status = INITIATED
⏳ Attempt 6: Status = INITIATED
⏳ Attempt 7: Status = INITIATED
⏳ Attempt 8: Status = INITIATED
⏳ Attempt 9: Status = INITIATED
⏳ Attempt 10: Status = INITIATED
⏳ Attempt 11: Status = INITIATED
⏳ Attempt 12: Status = INITIATED
⏳ Attempt 13: Status = INITIATED
⏳ Attempt 14: Status = INITIATED
⏳ Attempt 15: Status = INITIATED
⏳ Attempt 16: Status = INITIATED
⏳ Attempt 17: Status = INITIATED
⏳ Attempt 18: Status = INITIATED
⏳ Attempt 19: Status = INITIATED
⏳ Attempt 20: Status = INITIATED
⌛ Timeout waiting for connection.


## search bar 

In [19]:
import requests
import json

API_KEY = "ak_GepAqzYQ6d6WggBF_zot"  # 🔑 Your actual Composio API key
SEARCH_TERM = (input("please enter"))

url = "https://backend.composio.dev/api/v3/toolkits"

headers = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json"
}

params = {
    "limit": 100
}

response = requests.get(url, headers=headers, params=params)

if not response.ok:
    print("❌ Failed to fetch toolkits")
    print("Status:", response.status_code)
    print("Response:", response.text)
    exit()

try:
    data = response.json()
except Exception as e:
    print("❌ JSON parsing error:", e)
    print("Raw:", response.text)
    exit()

items = data.get("items", [])

# Filter and print full details of gmail
results = [
    item
    for item in items
    if SEARCH_TERM.lower() in item.get("name", "").lower()
    or SEARCH_TERM.lower() in item.get("slug", "").lower()
]

if not results:
    print(f"❌ No toolkit found for search term: {SEARCH_TERM}")
else:
    print(f"🔍 Found {len(results)} toolkit(s) for '{SEARCH_TERM}':\n")
    for r in results:
        print(json.dumps(r, indent=4))  # print full JSON details
        print("-" * 80)


🔍 Found 1 toolkit(s) for 'googlesheets':

{
    "name": "Googlesheets",
    "slug": "googlesheets",
    "auth_schemes": [
        "OAUTH2"
    ],
    "composio_managed_auth_schemes": [
        "OAUTH2"
    ],
    "is_local_toolkit": false,
    "meta": {
        "triggers_count": 2,
        "tools_count": 36,
        "description": "Google Sheets is a cloud-based spreadsheet tool enabling real-time collaboration, data analysis, and integration with other Google Workspace apps",
        "logo": "https://cdn.jsdelivr.net/gh/ComposioHQ/open-logos@master/google-sheets.svg",
        "app_url": "https://sheets.google.com/",
        "categories": [
            {
                "id": "productivity-&-project-management",
                "name": "productivity & project management"
            }
        ],
        "created_at": "2024-05-15T13:40:44.461Z",
        "updated_at": "2025-07-31T10:41:44.410Z"
    },
    "no_auth": false,
    "deprecated": {
        "toolkitId": "f034804e-ac1e-49ee-8e9d

## fetch all authentication configurations

In [20]:
import requests

def list_auth_configs(api_key: str):
    """
    List all authentication configurations from Composio API.

    :param api_key: Your Composio API key
    :return: JSON list of auth configs or None if request fails
    """
    url = "https://backend.composio.dev/api/v3/auth_configs"
    headers = {
        "x-api-key": api_key
    }

    try:
        response = requests.get(url, headers=headers)

        if response.ok:
            print("✅ Auth configs fetched successfully.")
            return response.json()
        else:
            print("❌ Failed to fetch auth configs")
            print("Status Code:", response.status_code)
            print("Response:", response.text)
            return None

    except Exception as e:
        print("⚠️ Error:", str(e))
        return None


# 🧪 Example usage
if __name__ == "__main__":
    YOUR_API_KEY = "ak_GepAqzYQ6d6WggBF_zot"  # <-- Replace with your API key
    configs = list_auth_configs(YOUR_API_KEY)
    print("Auth Configs:", configs)


✅ Auth configs fetched successfully.
Auth Configs: {'items': [{'id': 'ac_dAbunC3bfq5S', 'uuid': '2eafb2ed-ec5f-4f82-bc63-e0b9c8c03e1e', 'name': 'Googlesheets Auth Config', 'auth_scheme': 'OAUTH2', 'is_composio_managed': True, 'credentials': {'client_id': '****', 'client_secret': '****', 'scopes': ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/userinfo.email'], 'oauth_redirect_uri': 'https://backend.composio.dev/api/v1/auth-apps/add'}, 'created_by': 'fb674baf-882a-4460-8bb6-366181f0c34a', 'created_at': '2025-09-16T05:51:17.574Z', 'last_updated_at': '2025-09-16T05:51:17.574Z', 'status': 'ENABLED', 'toolkit': {'slug': 'googlesheets', 'logo': 'https://cdn.jsdelivr.net/gh/ComposioHQ/open-logos@master/google-sheets.svg'}, 'no_of_connections': 1, 'tool_access_config': {'tools_for_connected_account_creation': [], 'tools_available_for_execution': []}, 'type': 'default', 'deprecated_params': {'default_connector_id': 'tes

## fetch all authentication configurations in json

In [ ]:
import requests
import json

API_KEY = "ak_GepAqzYQ6d6WggBF_zot"
BASE_URL = "https://backend.composio.dev/api/v3"

headers = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json"
}

# Get all Auth Configs (isme connections count bhi hota hai)
resp = requests.get(f"{BASE_URL}/auth_configs", headers=headers)
resp.raise_for_status()

data = resp.json()
print(json.dumps(data, indent=2))


## fetch the details of a single auth config

In [16]:
import requests

def get_auth_config(api_key: str, nanoid: str):
    """
    Get a single authentication configuration by its nanoid.

    :param api_key: Your Composio API key
    :param nanoid: The unique ID of the auth config
    :return: JSON response with auth config details or None if failed
    """
    url = f"https://backend.composio.dev/api/v3/auth_configs/{nanoid}"
    headers = {
        "x-api-key": api_key
    }

    try:
        response = requests.get(url, headers=headers)

        if response.ok:
            print("✅ Auth config fetched successfully.")
            return response.json()
        else:
            print("❌ Failed to fetch auth config")
            print("Status Code:", response.status_code)
            print("Response:", response.text)
            return None

    except Exception as e:
        print("⚠️ Error:", str(e))
        return None


# 🧪 Example usage
if __name__ == "__main__":
    YOUR_API_KEY = "ak_GepAqzYQ6d6WggBF_zot"  # <-- Replace with your API key
    AUTH_CONFIG_ID = "ac_qFiXykHa--H4"   # <-- Replace this
    data = get_auth_config(YOUR_API_KEY, AUTH_CONFIG_ID)
    print("Auth Config Details:", data)


✅ Auth config fetched successfully.
Auth Config Details: {'id': 'ac_qFiXykHa--H4', 'uuid': '9129d671-2710-4c62-a1e4-dda0158a5dc4', 'name': 'auth_config_gmail_1757917191840', 'auth_scheme': 'OAUTH2', 'is_composio_managed': True, 'credentials': {'client_id': '****', 'client_secret': '****', 'oauth_redirect_uri': 'https://backend.composio.dev/api/v1/auth-apps/add'}, 'created_by': 'member', 'created_at': '2025-09-15T06:19:51.842Z', 'status': 'ENABLED', 'toolkit': {'slug': 'gmail', 'logo': 'https://cdn.jsdelivr.net/gh/ComposioHQ/open-logos@master/gmail.svg'}, 'last_updated_at': '2025-09-15T06:19:51.842Z', 'no_of_connections': 0, 'expected_input_fields': [], 'restrict_to_following_tools': [], 'type': 'default', 'tool_access_config': {'tools_for_connected_account_creation': [], 'tools_available_for_execution': []}, 'deprecated_params': {'default_connector_id': 'test-gmail-connector', 'member_uuid': 'fb674baf-882a-4460-8bb6-366181f0c34a', 'toolkit_id': 'a90e7d79-4f7a-4ff2-bd7d-19c78640b8f8', '

##  fetch the details of a single auth config in json 

In [ ]:
import requests
import json

API_KEY = "ak_GepAqzYQ6d6WggBF_zot"
BASE_URL = "https://backend.composio.dev/api/v3"

headers = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json"
}

# 🔹 Replace with your auth config ID (nanoid)
AUTH_CONFIG_ID = "ac_qFiXykHa--H4"

# Get details of a single Auth Config
resp = requests.get(f"{BASE_URL}/auth_configs/{AUTH_CONFIG_ID}", headers=headers)
resp.raise_for_status()

data = resp.json()
print(json.dumps(data, indent=2))
